# Inference

**Predict the timestamps for possible subtitles of your input audio**

In [5]:
import os
import warnings

import librosa
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from catalyst.dl import SupervisedRunner, CallbackOrder, Callback, CheckpointCallback
from fastprogress import progress_bar

In [4]:
!pip install fastprogress

In [6]:
from utils import check_type, extract_audio, mono_load, get_duration
from config import InferenceConfig as IC
from csrc.configurations import DatasetConfig as DC
from csrc.configurations import ModelConfig as MC

In [7]:
# For better debugging

os.environ["CUDA_LAUCH_BLOCKING"] = "1"

## User configuration/s

In [8]:
### IMPORTANT PARAM
PERIOD = 5 # 10, 5, 3, 2, 1
### IMPORTANT PARAM
THRESHOLD = 0.8 # 0.5, 0.9

In [9]:
# Those need no modification.

# Audio Sample rate. DO NOT change.
SR = DC.dataset_sample_rate

# Tag encoding. DO NOT change.
CODING = IC.coding_map

In [11]:
# Set and audiohandler your input file.

### Target file for inferencing.
TARGET_FILE_PATH = "machine-learning_speech-recognition_7601-291468-0006.wav"

TARGET_FILE_PATH = extract_audio(TARGET_FILE_PATH) if check_type(TARGET_FILE_PATH) else TARGET_FILE_PATH

### Output csv file name.
OUTPUT_FILE_NAME = "test"

OUTPUT_FILE = f"./inf/{OUTPUT_FILE_NAME}.csv"
OUTPUT_SOURCE_FILE = f"./inf/{OUTPUT_FILE_NAME}-all.csv"
if os.path.exists(OUTPUT_FILE):
    print("!Warning: Output file already exists, are you sure to rewrite the file?")

# Target model used for this prediction. Must be corresponding to the model during the training process.

### Set the model file path. The file path must be valid
MODEL_PATH = "./train/logs/sp2-32000hz/checkpoints/best.pth"

print(f"USING MODEL: {MODEL_PATH}")

USING MODEL: ./train/logs/sp2-32000hz/checkpoints/best.pth


## Dataset

In [12]:
from csrc.dataset import PANNsDataset

## Model

In [13]:
from csrc.models import PANNsCNN14Att, AttBlock

## Inference Settings

In [14]:
device = torch.device('cuda:0')

In [15]:
model = PANNsCNN14Att(**MC.sed_model_config)
model.att_block = AttBlock(2048, 2, activation='sigmoid')
model.att_block.init_weights()
model.load_state_dict(torch.load(MODEL_PATH)['model_state_dict'])
model.to(device)
model.eval()

FileNotFoundError: [Errno 2] No such file or directory: './train/logs/sp2-32000hz/checkpoints/best.pth'

In [16]:
y, _ = mono_load(TARGET_FILE_PATH)

Loading file: machine-learning_speech-recognition_7601-291468-0006.wav
Loading completed. Cost 1.12s



In [17]:
dataframe = pd.DataFrame()

## Prediction begin!

In [18]:
from utils import get_duration
audio_duration = get_duration(audio_file_path=TARGET_FILE_PATH, y=y, sr=SR)

print(f"Audio file duration: {audio_duration}s")

Audio file consistency ensured.
Audio file duration: 35.155s


In [22]:
audios = []

len_y = len(y)
start = 0
end = PERIOD * SR

# Split audio into clips.
while True:
    y_batch = y[start:end].astype(np.float32)
    if len(y_batch) != PERIOD * SR:
        y_pad = np.zeros(PERIOD * SR, dtype=np.float32)
        y_pad[:len(y_batch)] = y_batch
        audios.append(y_pad)
        break
    start = end
    end += PERIOD * SR
    audios.append(y_batch)

# Get tensors
arrays = np.asarray(audios)
tensors = torch.from_numpy(arrays)

estimated_event_list = []
global_time = 0.0
for image in progress_bar(tensors):
    image = image.view(1, image.size(0))
    image = image.to(device)

    with torch.no_grad():
        prediction = model(image)
        framewise_outputs = prediction["framewise_output"].detach(
            ).cpu().numpy()[0]
            
    thresholded = framewise_outputs >= THRESHOLD

    for target_idx in range(thresholded.shape[1]):
        if thresholded[:, target_idx].mean() == 0:
            pass
        else:
            detected = np.argwhere(thresholded[:, target_idx]).reshape(-1)
            head_idx = 0
            tail_idx = 0
            while True:
                if (tail_idx + 1 == len(detected)) or (
                        detected[tail_idx + 1] - 
                        detected[tail_idx] != 1):
                    onset = 0.01 * detected[
                        head_idx] + global_time
                    offset = 0.01 * detected[
                        tail_idx] + global_time
                    onset_idx = detected[head_idx]
                    offset_idx = detected[tail_idx]
                    max_confidence = framewise_outputs[
                        onset_idx:offset_idx, target_idx].max()
                    mean_confidence = framewise_outputs[
                        onset_idx:offset_idx, target_idx].mean()
                    estimated_event = {
                        "speech_recognition": CODING[target_idx],
                        "start": onset,
                        "end": offset,
                        "max_confidence": max_confidence,
                        "mean_confidence": mean_confidence,
                    }
                    estimated_event_list.append(estimated_event)
                    head_idx = tail_idx + 1
                    tail_idx = tail_idx + 1
                    if head_idx >= len(detected):
                        break
                else:
                    tail_idx += 1
    global_time += PERIOD
    
prediction_df = pd.DataFrame(estimated_event_list)

AssertionError: Torch not compiled with CUDA enabled

## Post process

In [20]:
# Secure output file offset: max offset should be less than audio duration.

max_offset = prediction_df.iloc[-1].end
if max_offset > audio_duration:
    prediction_df.iloc[-1].end = audio_duration

NameError: name 'prediction_df' is not defined

In [21]:
prediction_df[prediction_df.speech_recognition=="speech"].to_csv(OUTPUT_FILE, index=False)
prediction_df.to_csv(OUTPUT_SOURCE_FILE, index=False)

NameError: name 'prediction_df' is not defined